<a href="https://colab.research.google.com/github/chenkaiwen111811/PL-Repo/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1h6zobvpt8Ra91J3RYsD49yDSUvQmJvKEl1otWGfmZvQ/edit?usp=sharing"
WORKSHEET_NAME = "HW6"
TIMEZONE = "Asia/Taipei"
SEMESTER_START = "2025-09-01"  # 學期開始日期

In [5]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('gemini')

genai.configure(api_key=GEMINI_API_KEY)

In [6]:
!pip install pyPDF2

In [7]:
import PyPDF2

In [8]:
class CourseManager:
    def __init__(self):
        self.schedule_data = []
        self.course_details = {}
        self.weekly_schedule = []


    def parse_course_outline(self, pdf_path, course_name):
        """解析課程綱要 - 提取結構化資料"""
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in reader.pages:
                full_text += page.extract_text()

        # 提取基本資訊
        basic_info = self._extract_basic_info(full_text)

        # 提取教學進度
        schedule = self._extract_teaching_schedule(full_text)

        # 提取評量方式
        grading = self._extract_grading(full_text)

        # 提取參考書目
        references = self._extract_references(full_text)

        self.course_details[course_name] = {
            'basic_info': basic_info,
            'schedule': schedule,
            'grading': grading,
            'references': references,
            'full_text': full_text
        }

        return self.course_details[course_name]

    def _extract_basic_info(self, text):
        """提取基本資訊"""
        info = {}

        # 課程名稱
        course_name_match = re.search(r'課程名稱\s*(.+?)(?:\n|英文名稱)', text)
        if course_name_match:
            info['課程名稱'] = course_name_match.group(1).strip()

        # 英文名稱
        eng_name_match = re.search(r'英文名稱\s*(.+?)(?:\n|全/半年)', text)
        if eng_name_match:
            info['英文名稱'] = eng_name_match.group(1).strip()

        # 學分數
        credits_match = re.search(r'學分數\s*(\d+\.?\d*)', text)
        if credits_match:
            info['學分數'] = credits_match.group(1)

        # 授課教師
        teacher_match = re.search(r'授課教師\s*(.+?)(?:\n|教學進度)', text)
        if teacher_match:
            info['授課教師'] = teacher_match.group(1).strip()

        # 課程簡介
        intro_match = re.search(r'課程簡介\s*(.+?)(?:\n課程目標|$)', text, re.DOTALL)
        if intro_match:
            info['課程簡介'] = intro_match.group(1).strip()[:500]

        return info

    def _extract_teaching_schedule(self, text):
        """提取每週進度"""
        schedule = []

        # 匹配週次的多種格式
        patterns = [
            r'第(\d+)週[：:\s]*(.+?)(?=第\d+週|教學方法|評量方法|$)',
            r'Week\s*(\d+)[：:\s]*(.+?)(?=Week|教學方法|評量方法|$)',
            r'(\d+)\.\s*(.+?)(?=\n\d+\.|教學方法|評量方法|$)',
        ]

        for pattern in patterns:
            matches = list(re.finditer(pattern, text, re.DOTALL | re.IGNORECASE))
            if matches:
                for match in matches:
                    week_num = int(match.group(1))
                    content = match.group(2).strip()

                    # 清理內容
                    content = re.sub(r'\s+', ' ', content)  # 合併空白
                    content = content[:300]  # 限制長度

                    if content and len(content) > 5:  # 過濾太短的內容
                        schedule.append({
                            'week': week_num,
                            'content': content
                        })
                break  # 找到一種格式就停止

        # 如果沒找到，嘗試找課程大綱
        if not schedule:
            outline_match = re.search(r'(課程內容|教學進度與主題)(.+?)(?=教學方法|評量方法|$)', text, re.DOTALL)
            if outline_match:
                content = outline_match.group(2)
                lines = [l.strip() for l in content.split('\n') if l.strip()]

                for idx, line in enumerate(lines[:16], 1):
                    if len(line) > 10:
                        schedule.append({
                            'week': idx,
                            'content': line[:300],
                            'is_guessed': True
                        })

        return sorted(schedule, key=lambda x: x['week'])

    def _extract_grading(self, text):
        """提取評量方式"""
        grading = []

        # 尋找評量區塊
        grading_match = re.search(r'評量方法(.+?)(?=參考書目|教學方法|$)', text, re.DOTALL)
        if grading_match:
            grading_text = grading_match.group(1)

            # 提取各項評量
            items = re.findall(r'(期中考|期末考|作業|報告|出席|課堂討論參與|其他|成果展覽).*?(\d+)\s*%', grading_text)

            for item, percentage in items:
                grading.append({
                    '項目': item,
                    '百分比': int(percentage)
                })

        return grading

    def _extract_references(self, text):
        """提取參考書目"""
        references = []

        ref_match = re.search(r'參考書目(.+?)(?=版權所有|$)', text, re.DOTALL)
        if ref_match:
            ref_text = ref_match.group(1)

            # 分割成多行
            lines = [l.strip() for l in ref_text.split('\n') if l.strip()]

            for line in lines[:20]:  # 最多20本
                if len(line) > 10 and not line.startswith('參考書'):
                    references.append(line)

        return references

    def convert_to_dataframe(self, course_name):
        """將課程資訊轉換為DataFrame"""
        details = self.course_details.get(course_name, {})

        # 基本資訊 DataFrame
        basic_df = pd.DataFrame([details['basic_info']]) if details.get('basic_info') else pd.DataFrame()

        # 教學進度 DataFrame
        schedule_df = pd.DataFrame(details['schedule']) if details.get('schedule') else pd.DataFrame()

        # 評量方式 DataFrame
        grading_df = pd.DataFrame(details['grading']) if details.get('grading') else pd.DataFrame()

        # 參考書目 DataFrame
        references_df = pd.DataFrame({'參考書目': details['references']}) if details.get('references') else pd.DataFrame()

        return {
            'basic': basic_df,
            'schedule': schedule_df,
            'grading': grading_df,
            'references': references_df
        }

In [9]:
class SheetsManager:
    def __init__(self, sheet_url=SHEET_URL):
        self.spreadsheet = gc.open_by_url(sheet_url)

    def create_or_get_worksheet(self, title, rows=100, cols=20):
        """建立或取得工作表"""
        try:
            worksheet = self.spreadsheet.worksheet(title)
            print(f"📄 工作表 '{title}' 已存在")
        except:
            worksheet = self.spreadsheet.add_worksheet(title=title, rows=rows, cols=cols)
            print(f"✅ 建立新工作表 '{title}'")

        return worksheet

    def write_dataframe(self, worksheet_title, df, clear=True):
        """寫入 DataFrame 到工作表"""
        worksheet = self.create_or_get_worksheet(worksheet_title)

        if clear:
            worksheet.clear()

        if not df.empty:
            set_with_dataframe(worksheet, df, include_index=False)
            print(f"✅ 已寫入 {len(df)} 筆資料到 '{worksheet_title}'")
        else:
            print(f"⚠️ '{worksheet_title}' 沒有資料可寫入")

        return worksheet

    def write_course_outline(self, course_name, course_dfs):
        """寫入完整課程綱要到分頁"""
        sheet_name = f"【綱要】{course_name}"
        worksheet = self.create_or_get_worksheet(sheet_name, rows=200, cols=10)
        worksheet.clear()

        # 組合所有資訊
        current_row = 1

        # 1. 基本資訊
        if not course_dfs['basic'].empty:
            worksheet.update_cell(current_row, 1, "📋 基本資訊")
            worksheet.format(f"A{current_row}:J{current_row}", {
                "backgroundColor": {"red": 0.2, "green": 0.6, "blue": 0.9},
                "textFormat": {"bold": True, "fontSize": 12}
            })
            current_row += 1

            set_with_dataframe(worksheet, course_dfs['basic'],
                             row=current_row, include_index=False)
            current_row += len(course_dfs['basic']) + 2

        # 2. 教學進度
        if not course_dfs['schedule'].empty:
            worksheet.update_cell(current_row, 1, "📅 教學進度")
            worksheet.format(f"A{current_row}:J{current_row}", {
                "backgroundColor": {"red": 0.3, "green": 0.7, "blue": 0.5},
                "textFormat": {"bold": True, "fontSize": 12}
            })
            current_row += 1

            set_with_dataframe(worksheet, course_dfs['schedule'],
                             row=current_row, include_index=False)
            current_row += len(course_dfs['schedule']) + 2

        # 3. 評量方式
        if not course_dfs['grading'].empty:
            worksheet.update_cell(current_row, 1, "📊 評量方式")
            worksheet.format(f"A{current_row}:J{current_row}", {
                "backgroundColor": {"red": 0.9, "green": 0.7, "blue": 0.3},
                "textFormat": {"bold": True, "fontSize": 12}
            })
            current_row += 1

            set_with_dataframe(worksheet, course_dfs['grading'],
                             row=current_row, include_index=False)
            current_row += len(course_dfs['grading']) + 2

        # 4. 參考書目
        if not course_dfs['references'].empty:
            worksheet.update_cell(current_row, 1, "📚 參考書目")
            worksheet.format(f"A{current_row}:J{current_row}", {
                "backgroundColor": {"red": 0.7, "green": 0.5, "blue": 0.8},
                "textFormat": {"bold": True, "fontSize": 12}
            })
            current_row += 1

            set_with_dataframe(worksheet, course_dfs['references'],
                             row=current_row, include_index=False)

        print(f"✅ 完整課程綱要已寫入 '{sheet_name}'")
        return worksheet



In [10]:
def generate_ai_reminder(course_name, week_content, week_num, grading_info):
    """使用 Gemini 生成個性化提醒"""

    grading_text = "、".join([f"{g['項目']}{g['百分比']}%" for g in grading_info]) if grading_info else "無資料"

    prompt = f"""
你是一位溫暖貼心的學習助理 🌟

請根據以下課程資訊，生成一則**簡潔、實用、有溫度**的學習提醒（約100-150字）。

**課程**: {course_name}
**第 {week_num} 週**
**主題**: {week_content}
**評量**: {grading_text}

請用繁體中文，包含：
1. 一句鼓勵開場（加emoji）
2. 本週核心重點（1-2句）
3. 實用建議（如：預習、複習、準備作業等）

語氣要親切、正面、激勵人心！
"""

    try:
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"⚠️ AI生成失敗: {e}")
        return f"📚 第{week_num}週：{week_content[:80]}..."

In [11]:
def generate_weekly_suggestions(course_manager, semester_start=SEMESTER_START):
    """生成完整學期建議"""
    start_date = dt.strptime(semester_start, '%Y-%m-%d')
    suggestions = []

    for course_name, details in course_manager.course_details.items():
        schedule = details.get('schedule', [])
        grading = details.get('grading', [])

        for week_info in schedule:
            week_num = week_info['week']
            week_date = start_date + timedelta(weeks=week_num - 1)

            # 使用 AI 生成提醒
            ai_reminder = generate_ai_reminder(
                course_name,
                week_info['content'],
                week_num,
                grading
            )

            suggestions.append({
                '日期': week_date.strftime('%Y-%m-%d'),
                '星期': week_date.strftime('%A'),
                '週次': week_num,
                '課程': course_name,
                '主題': week_info['content'][:150],
                'AI學習建議': ai_reminder,
                '是否推測': week_info.get('is_guessed', False)
            })

    df = pd.DataFrame(suggestions).sort_values(['日期', '課程'])
    return df


# ============ 主流程 ============
def main_process():
    """主要處理流程"""

    print("=" * 60)
    print("📚 智慧課程管理系統")
    print("=" * 60)

    # 1. 上傳檔案
    from google.colab import files
    print("\n📤 請上傳所有 PDF 檔案...")
    uploaded = files.upload()

    print(f"\n✅ 已上傳 {len(uploaded)} 個檔案")

    # 2. 初始化
    course_manager = CourseManager()
    sheets_manager = SheetsManager(SHEET_URL)

    # 3. 處理課表
    print("\n" + "=" * 60)
    print("📋 步驟 1: 處理課表")
    print("=" * 60)

    schedule_pdf = [f for f in uploaded.keys()
                   if 'CourseSchedule' in f or '課表' in f or 'schedule' in f.lower()]

    if schedule_pdf:
        print(f"處理檔案: {schedule_pdf[0]}")

        # --- 修改開始 ---
        print("正在從 CSV 讀取課表...")
        schedule_df = pd.read_csv(schedule_pdf[0])
        # (我們不再需要 extract_schedule_from_pdf 和 save_schedule_to_csv)
        # --- 修改結束 ---

        # 寫入 Google Sheets
        sheets_manager.write_dataframe("【課表】", schedule_df)
        print(f"\n課表預覽 ({len(schedule_df)} 筆時段):") # 這行可以印出來確認
        print(schedule_df.to_string())                 # 這行可以印出來確認

    # 4. 處理課程綱要
    print("\n" + "=" * 60)
    print("📖 步驟 2: 處理課程綱要")
    print("=" * 60)

    course_pdfs = {
    '機械製造': ['機械製造', '金工'],
    '木工製造': ['木工製造', '木工'],
    '色彩學': ['色彩學'],
    '教師專業發展': ['教師專業發展', '教師專業'],
    '班級經營': ['班級經營', '班經'],
    '學習評量': ['學習評量'],
    '程式語言': ['程式語言'],
    '科技系統與社會發展': ['科技系統'],
    '電腦輔助製圖': ['電腦輔助製圖', '電腦製圖'],
    '機構設計': ['機構設計']
}
    processed_courses = []

    for course_name, keywords in course_pdfs.items():
        matching_files = [f for f in uploaded.keys()
                         if any(kw in f for kw in keywords) and f not in schedule_pdf]

        if matching_files:
            print(f"\n處理: {course_name} ({matching_files[0]})")
            course_manager.parse_course_outline(matching_files[0], course_name)

            # 轉換為 DataFrame
            course_dfs = course_manager.convert_to_dataframe(course_name)

            # 寫入 Google Sheets（獨立分頁）
            sheets_manager.write_course_outline(course_name, course_dfs)

            processed_courses.append(course_name)
        else:
            print(f"⚠️ 未找到 {course_name} 的 PDF")

    print(f"\n✅ 成功處理 {len(processed_courses)} 門課程")

    # 5. 生成 AI 學習建議
    print("\n" + "=" * 60)
    print("🤖 步驟 3: 生成 AI 學習建議")
    print("=" * 60)

    suggestions_df = generate_weekly_suggestions(course_manager)
    suggestions_df.to_csv('learning_suggestions.csv', index=False, encoding='utf-8-sig')

    # 寫入 Google Sheets
    sheets_manager.write_dataframe("【AI學習建議】", suggestions_df)

    print(f"\n✅ 共生成 {len(suggestions_df)} 則學習建議")
    print("\n前 5 則建議預覽:")
    print(suggestions_df.head().to_string())

    # 6. 下載檔案
    print("\n" + "=" * 60)
    print("💾 下載生成的檔案")
    print("=" * 60)

    if os.path.exists('course_schedule.csv'):
        files.download('course_schedule.csv')
        print("✅ 已下載: course_schedule.csv")

    files.download('learning_suggestions.csv')
    print("✅ 已下載: learning_suggestions.csv")

    # 7. 總結
    print("\n" + "=" * 60)
    print("📊 處理完成總結")
    print("=" * 60)
    print(f"✅ Google Sheets 分頁結構:")
    print(f"   - 【課表】: {len(schedule_df)} 筆資料")
    print(f"   - 【AI學習建議】: {len(suggestions_df)} 筆資料")
    for course_name in processed_courses:
        print(f"   - 【綱要】{course_name}")

    print(f"\n🔗 Google Sheets 連結:")
    print(f"   {SHEET_URL}")

    return course_manager, sheets_manager, schedule_df, suggestions_df

In [12]:
if __name__ == "__main__":
    # 檢查並安裝套件
    print("📦 檢查必要套件...")
    try:
        import PyPDF2
    except:
        os.system("pip install -q PyPDF2")

    # 執行主流程
    course_manager, sheets_manager, schedule_df, suggestions_df = main_process()

    print("\n🎉 所有處理完成！請前往 Google Sheets 查看結果")

📦 檢查必要套件...
📚 智慧課程管理系統

📤 請上傳所有 PDF 檔案...


Saving 木工製造.pdf to 木工製造 (3).pdf
Saving 色彩學.pdf to 色彩學 (3).pdf
Saving 科技系統.pdf to 科技系統 (3).pdf
Saving 班級經營.pdf to 班級經營 (3).pdf
Saving 教師專業發展.pdf to 教師專業發展 (3).pdf
Saving 程式語言.pdf to 程式語言 (3).pdf
Saving 電腦輔助製圖.pdf to 電腦輔助製圖 (3).pdf
Saving 課表.csv to 課表 (3).csv
Saving 學習評量.pdf to 學習評量 (3).pdf
Saving 機械製造.pdf to 機械製造 (3).pdf
Saving 機構設計.pdf to 機構設計 (3).pdf

✅ 已上傳 11 個檔案

📋 步驟 1: 處理課表
處理檔案: 課表 (3).csv
正在從 CSV 讀取課表...
📄 工作表 '【課表】' 已存在
✅ 已寫入 28 筆資料到 '【課表】'

課表預覽 (28 筆時段):
         課程名稱 星期  節次           時間          教室
0        機械製造  一  01  08:10-09:00  科技系TC504教室
1        機械製造  一  02  09:10-10:00  科技系TC504教室
2        機械製造  一  03  10:20-11:10  科技系TC504教室
3        機械製造  一  04  11:20-12:10  科技系TC504教室
4        木工製造  二  01  08:10-09:00  科技系TC501教室
5        木工製造  二  02  09:10-10:00  科技系TC501教室
6        木工製造  二  03  10:20-11:10  科技系TC501教室
7        木工製造  二  04  11:20-12:10  科技系TC501教室
8         色彩學  二  06  13:20-14:10  科技系TB213教室
9         色彩學  二  07  14:20-15:10  科技系TB213教室
10        色彩學  二  08  15:

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 711.61ms


⚠️ AI生成失敗: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 36.530179412s.


⚠️ AI生成失敗: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 36.234435788s.


...

<IPython.core.display.Javascript object>

✅ 已下載: course_schedule.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 已下載: learning_suggestions.csv

📊 處理完成總結
✅ Google Sheets 分頁結構:
   - 【課表】: 28 筆資料
   - 【AI學習建議】: 140 筆資料
   - 【綱要】機械製造
   - 【綱要】木工製造
   - 【綱要】色彩學
   - 【綱要】教師專業發展
   - 【綱要】班級經營
   - 【綱要】學習評量
   - 【綱要】程式語言
   - 【綱要】科技系統與社會發展
   - 【綱要】電腦輔助製圖
   - 【綱要】機構設計

🔗 Google Sheets 連結:
   https://docs.google.com/spreadsheets/d/1h6zobvpt8Ra91J3RYsD49yDSUvQmJvKEl1otWGfmZvQ/edit?usp=sharing

🎉 所有處理完成！請前往 Google Sheets 查看結果
